In [18]:
import os

from pyciemss.ODE.base import BetaNoisePetriNetODESystem
from pyciemss.ODE.events import Event, StartEvent, LoggingEvent, ObservationEvent, StaticParameterInterventionEvent
import pyciemss
import torch

In [3]:
STARTERKIT_PATH = "../test/models/starter_kit_examples/"
MIRA_PATH = "../test/models/evaluation_examples/"

filename = "CHIME-SIR/model_petri.json"
filename = os.path.join(STARTERKIT_PATH, filename)

model = BetaNoisePetriNetODESystem.from_mira(filename)

start_event = StartEvent(0.0, {"S": 0.99, "I": 0.01, "R": 0.0})
model.load_event(start_event)

tspan = range(1, 10)
logging_events = [LoggingEvent(t) for t in tspan]
model.load_events(logging_events)

solution = model()

# See that the solution returns a dictionary where each value has length 10, one for each logging event.
assert len(solution["I"]) == len(solution["R"]) == len(solution["S"]) == len(tspan)

solution

{'I': tensor([0.0097, 0.0093, 0.0089, 0.0084, 0.0079, 0.0074, 0.0068, 0.0063, 0.0058]),
 'R': tensor([0.0096, 0.0188, 0.0276, 0.0360, 0.0439, 0.0513, 0.0582, 0.0646, 0.0704]),
 'S': tensor([0.9807, 0.9719, 0.9635, 0.9556, 0.9482, 0.9413, 0.9350, 0.9291, 0.9238])}

In [4]:
# Remove logging events
model.remove_logging_events()

# Add observations
observation_events = [ObservationEvent(1.1, {"S": 0.9, "I": 0.09, "R": 0.01}), 
                      ObservationEvent(2.1, {"S": 0.8, "I": 0.18, "R": 0.02}),
                      ObservationEvent(3.1, {"S": 0.7, "I": 0.27, "R": 0.03}),
                      ObservationEvent(4.1, {"S": 0.6, "I": 0.36, "R": 0.04})]

model.load_events(observation_events)

In [5]:
model._static_events

[StartEvent(time=0.0, initial_state={'S': tensor(0.9900), 'I': tensor(0.0100), 'R': tensor(0.)}),
 ObservationEvent(time=1.100000023841858, observation={'S': tensor(0.9000), 'I': tensor(0.0900), 'R': tensor(0.0100)}),
 ObservationEvent(time=2.0999999046325684, observation={'S': tensor(0.8000), 'I': tensor(0.1800), 'R': tensor(0.0200)}),
 ObservationEvent(time=3.0999999046325684, observation={'S': tensor(0.7000), 'I': tensor(0.2700), 'R': tensor(0.0300)}),
 ObservationEvent(time=4.099999904632568, observation={'S': tensor(0.6000), 'I': tensor(0.3600), 'R': tensor(0.0400)})]

In [6]:
# Show that inference works.

from pyro.infer.autoguide import AutoNormal
from pyro.poutine import block
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro

guide = AutoNormal(model)
optim = Adam({'lr': 0.03})
loss_f = Trace_ELBO(num_particles=1)
verbose = True

svi = SVI(model, guide, optim, loss=loss_f)

pyro.clear_param_store()

for j in range(100):
    # calculate the loss and take a gradient step
    # Passing a data argument to svi.step() will condition the model on the data.
    loss = svi.step()
    if verbose:
        if j % 25 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss))

[iteration 0001] loss: 20.1702
[iteration 0026] loss: 22.6139
[iteration 0051] loss: 19.8579
[iteration 0076] loss: 20.5666


In [7]:
# Remove the observation events
model.remove_observation_events()

In [8]:
# Load some static parameter intervention events
model.load_event(StaticParameterInterventionEvent(2.99, "beta", 0.0))
model.load_event(StaticParameterInterventionEvent(4.11, "beta", 10.))

# Load the logging events again
model.load_events(logging_events)

In [9]:
model._static_events

[StartEvent(time=0.0, initial_state={'S': tensor(0.9900), 'I': tensor(0.0100), 'R': tensor(0.)}),
 LoggingEvent(time=1),
 LoggingEvent(time=2),
 StaticParameterInterventionEvent(time=2.990000009536743, parameter=beta, value=0.0),
 LoggingEvent(time=3),
 LoggingEvent(time=4),
 StaticParameterInterventionEvent(time=4.110000133514404, parameter=beta, value=10.0),
 LoggingEvent(time=5),
 LoggingEvent(time=6),
 LoggingEvent(time=7),
 LoggingEvent(time=8),
 LoggingEvent(time=9)]

In [10]:
model._observation_var_names

[]

In [11]:
# use pyro predictive
from pyro.infer import Predictive

# Get the return value of model.
predictions = Predictive(model, guide=guide, num_samples=10)()

In [12]:
predictions

{'a_beta': tensor([1.0564, 1.0765, 1.0672, 1.0678, 1.0245, 1.0540, 1.0822, 1.0614, 1.0775,
         1.0659]),
 'a_gamma': tensor([0.9383, 0.9727, 0.9384, 0.9148, 0.9471, 0.9540, 0.9223, 0.9151, 0.9537,
         0.9377]),
 'I_sol': tensor([[0.0111, 0.0121, 0.0129, 0.0051, 0.6475, 0.2977, 0.1169, 0.0458, 0.0179],
         [0.0109, 0.0118, 0.0124, 0.0047, 0.6390, 0.2889, 0.1096, 0.0415, 0.0157],
         [0.0112, 0.0124, 0.0133, 0.0052, 0.6468, 0.2966, 0.1164, 0.0456, 0.0178],
         [0.0115, 0.0130, 0.0143, 0.0057, 0.6515, 0.3015, 0.1211, 0.0486, 0.0195],
         [0.0106, 0.0112, 0.0115, 0.0045, 0.6471, 0.2991, 0.1164, 0.0452, 0.0175],
         [0.0109, 0.0117, 0.0123, 0.0047, 0.6442, 0.2948, 0.1140, 0.0439, 0.0169],
         [0.0115, 0.0131, 0.0146, 0.0058, 0.6489, 0.2985, 0.1190, 0.0474, 0.0188],
         [0.0114, 0.0128, 0.0141, 0.0056, 0.6519, 0.3021, 0.1213, 0.0486, 0.0195],
         [0.0111, 0.0122, 0.0131, 0.0051, 0.6430, 0.2925, 0.1131, 0.0436, 0.0168],
         [0.0112, 0.012

In [13]:
model()

{'I': tensor([0.0094, 0.0088, 0.0081, 0.0030, 0.6284, 0.2923, 0.1066, 0.0388, 0.0141]),
 'R': tensor([0.0099, 0.0191, 0.0278, 0.0330, 0.2222, 0.7065, 0.8932, 0.9611, 0.9858]),
 'S': tensor([9.8070e-01, 9.7203e-01, 9.6409e-01, 9.6409e-01, 1.4935e-01, 1.2540e-03,
         1.9866e-04, 1.0157e-04, 7.9596e-05])}

In [19]:
# SCAFFOLDING FOR DYNAMIC EVENT HANDLING BELOW

import pyro
from torch import Tensor
from torchdiffeq import odeint_event, odeint

Time = Tensor
State = tuple[Tensor, ...]

class BaseODEModel(pyro.nn.PyroModule):

    def __init__(self, static_events: list[Event]):
        super().__init__()
        # This is a list of events that are always sorted by time.
        # TODO: probably pre-sort this list just in case.
        self.static_events = static_events

    def deriv(self, t: Time, state: State) -> State:
        raise NotImplementedError

    @pyro.nn.pyro_method
    def param_prior(self):
        raise NotImplementedError

    @pyro.nn.pyro_method
    def initial_conditions_prior(self):
        raise NotImplementedError

    # @pyro.nn.pyro_method
    # def observation_model(self, state: State, tspan: Time, ?) -> ?: …

    def solve(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        
        current_state = initial_state
        current_time = initial_time

        solution = torch.tensor([])

        for i, static_event in enumerate(self.static_events):
            
            # TODO: change below
            # Note: Immediate goal is to get self.solve() to generate ode solutions that stop at the event times.
            # Note: Next we will add log likelihood at those events using an observation model.

            # Note: each static_event is an Event object, which has a forward() method that returns a Tensor.
            # TODO: Chad, could you please make this actually make sense.
            event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)
    
            # TODO: Add log likelihood for event_solution.
            # Note: This will be done by adding an observation model to the ODE model and calling it with the event_solution as an argument.

            solution = torch.cat([solution, event_solution], dim=-1)
            # current = event_solution[-1]


    def forward(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        # Sample parameters from the prior. These parameters are generated as attributes of the model.
        self.param_prior()

        # TODO: Sample initial conditions from the prior instead of taking them as input.

        # Solve the ODE, taking into account any intervention and conditioning events.
        return self.solve(initial_state, initial_time)




In [16]:
event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)

NameError: name 'self' is not defined

In [ ]:

# TODO: Chad, initialize a stop event here.
# stop_event = ...

# TODO: Chad, create a model the subclasses BaseODEModel and has all of the above missing methods implemented.
# deriv, param_prior, initial_conditions_prior, observation_model
# class ActualODEModel(BaseODEModel):
#   ...

# TODO: Chad, instantiate the model here.
# model = ActualODEModel([stop_event])

# TODO: Chad, Call model.forward() here.
# solution, _ = model(initial_state, tspan)

In [20]:
observations = [ObservationEvent(torch.tensor([2.1]), {"I": torch.tensor([0.1])})]


In [22]:
conditioned_model = pyciemss.observe(model, observations)

AttributeError: module 'pyciemss' has no attribute 'observe'